In [ ]:
# Validate the Model on the Hub Cluster - Federated Learning

- **Current Cluster:** Hub cluster
- **Dataset:** MNIST (Digits 0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [ ]:
# check the models
!ls -la /data/models

In [ ]:
import os
os.environ["PATH"] += ":/home/jovyan/.local/bin"

!pip install -q torch
!pip install -q torchvision==0.20.0

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.optim.lr_scheduler import StepLR

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
model = Net()
network_state_dict = torch.load("/data/models/model_2025-02-07-00-47-01.pkl_round_10.pth", weights_only=True)
model.load_state_dict(network_state_dict)

In [ ]:
transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = torchvision.datasets.MNIST('./data', train=True, download=True, transform=transform)
dataset2 = torchvision.datasets.MNIST('./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size = 1000, shuffle=True)

In [ ]:
def plot_distribution(dataset, title):
    import matplotlib.pyplot as plt
    import numpy as np
    
    labels = [data[1] for data in dataset]
    unique_labels, label_counts = torch.unique(torch.tensor(labels), return_counts=True)
    
    plt.figure(figsize=(4, 2))

    counts_dict = {
        label.item(): count.item() for label, count in zip(unique_labels, label_counts)
    }

    all_labels = np.arange(10)
    all_label_counts = [counts_dict.get(label, 0) for label in all_labels]

    plt.bar(all_labels, all_label_counts)
    plt.title(title)
    plt.xlabel("Digit")
    plt.ylabel("Count")
    plt.xticks(all_labels)
    plt.show()
    
plot_distribution(dataset2, "test dataset distribution")

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

In [ ]:
device = torch.device("cpu")
test(model, device, test_loader)

In [ ]:
# confusion matrix

def compute_confusion_matrix(model, testset):
    import numpy as np

    # Initialize lists to store true labels and predicted labels
    true_labels = []
    predicted_labels = []

    # Iterate over the test set to get predictions
    for image, label in testset:
        # Forward pass through the model to get predictions
        output = model(image.unsqueeze(0))  # Add batch dimension
        _, predicted = torch.max(output, 1)

        # Append true and predicted labels to lists
        true_labels.append(label)
        predicted_labels.append(predicted.item())

    # Convert lists to numpy arrays
    true_labels = np.array(true_labels)
    predicted_labels = np.array(predicted_labels)

    # Compute confusion matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    return cm


def plot_confusion_matrix(cm, title):
    import matplotlib.pyplot as plt
    import seaborn as sns

    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", linewidths=0.5)
    plt.title(title)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

In [ ]:
confusion_matrix_model_all = compute_confusion_matrix(model, dataset2)
plot_confusion_matrix(confusion_matrix_model_all, "model")

In [ ]:
# plot samples

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

batch_idx, example_data.shape, example_targets.shape, len(example_targets)

In [ ]:
with torch.no_grad():
  output = model(example_data)
print(output.shape)

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12, 4))
rows, cols = 2, 5  # Arrange in a 2-row, 5-column grid
for i in range(10):
  plt.subplot(rows, cols, i + 1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Prediction: {}".format(output.data.max(1, keepdim=True)[1][i].item()))
  plt.xticks([])
  plt.yticks([])